# Weekly activity
1. Rotate image by 45 degrees without cropping the sides of the image. (Hint: There are 2 strategies to tackle these problems). Use _"lena.jfif"_ as the input image.
    - Use external libraries `imutils`.  
    - Modify the transformation matrix.
2. Use the images with titles: _"flower.jfif"_ and _"native-bee.png"_. I want to put flower above an image. If I add two images, it will change color. If I blend it, I get a transparent effect. But I want it to be opaque. If it was a rectangular region, we could use the ROI as we did in the previous section. But flower is not a rectangular region. This is where bitwise operations, like AND, OR, NOT and XOR really come in handy. The associated functions are `cv.bitwise_and()`, `cv.bitwise_or()` and `cv.bitwise_not()`. You need to use `cv.threshold` function to segment the flower. Please refer to [online documentation](https://docs.opencv.org/4.x/d0/d86/tutorial_py_image_arithmetics.html) for more info. The result should resemble the following:  
![bee and flowers](img_embed/activity3.PNG "bee_flower")
3. Write a function that randomly crop the central region of an image. The method signature should be as shown in the following:
```
random_center_crop(image, min_crop_ratio, max_crop_ratio)
```

4. Aside from Gaussian noise, name another common type of noise. Write the code to demonstrate how the noise can be included in an image.

In [4]:
import cv2 as cv
import imutils
import numpy as np

from utils import display_image

**Question 1**

In [15]:
image = cv.imread("images/lena.jfif")

In [42]:
# Method 1
img_rotated = imutils.rotate_bound(image, 45)

display_image("rotated image (imutils)", img_rotated)

In [43]:
# Method 2
def rotate_image_no_crop(image, angle):
    height, width = image.shape[:2]

    # calculate rotation parameters
    theta = np.radians(angle)
    cos_theta, sin_theta = np.cos(theta), np.sin(theta)

    # determine new width and height of image
    new_width = int(width * abs(cos_theta) + height * abs(sin_theta))
    new_height = int(width * abs(sin_theta) + height * abs(cos_theta))

    # create a larger canvas and center the image
    enlarged_image = np.zeros((new_height, new_width, 3), dtype=np.uint8)
    offset_x, offset_y = (new_width - width) // 2, (new_height - height) // 2
    enlarged_image[offset_y:offset_y + height, offset_x:offset_x + width] = image
    
    # find center of new image and calculate rotation matrix
    cx, cy = new_width // 2, new_height // 2
    rotation_matrix = np.array([[cos_theta, -sin_theta, cx * (1 - cos_theta) + cy * sin_theta],
                                [sin_theta, cos_theta, cy * (1 - cos_theta) - cx * sin_theta]])

    # apply transformation
    rotated_image = cv.warpAffine(enlarged_image, rotation_matrix, (new_width, new_height))

    return rotated_image

rotated_image = rotate_image_no_crop(image, 45)

display_image("rotated image (transformation matrix", rotated_image)

**Question 2**

In [60]:
img = cv.imread("images/flower.jfif")
target_img = cv.imread("images/native-bee.png")
img.shape

(183, 275, 3)

In [64]:
enlarged_img = np.zeros((target_img.shape[0], target_img.shape[1], 3), dtype=np.uint8)
enlarged_img[0:img.shape[0], 0:img.shape[1]] = img
    
src_mask = cv.cvtColor(enlarged_img, cv.COLOR_BGR2GRAY)
_, src_mask = cv.threshold(src_mask, 1, 255, cv.THRESH_BINARY)

src_mask_inv = cv.bitwise_not(src_mask)

roi = cv.bitwise_and(enlarged_img, enlarged_img, src_mask)

overlay = cv.addWeighted(target_img, 1, roi, 1, 0)

display_image("overlay", overlay)

**Question 3**

In [2]:
def random_center_crop(img, min_crop_ratio, max_crop_ratio):
    if not (0.0 <= min_crop_ratio <= max_crop_ratio <= 1.0):
        raise ValueError("Invalid crop ratios. Ensure 0.0 <= min_crop_ratio <= max_crop_ratio <= 1.0.")

    height, width = img.shape[:2]
    crop_size = np.random.uniform(min_crop_ratio, max_crop_ratio)

    crop_h, crop_w = int(height * crop_size), int(width * crop_size)
    offset_x = (width - crop_w) // 2
    offset_y = (height - crop_h) // 2

    cropped_img = img[offset_y:offset_y + crop_h, offset_x:offset_x + crop_w]
    display_image("random crop", cropped_img)

In [13]:
# Example usage:
input_image = cv.imread("images/lena.png")
cropped_image = random_center_crop(input_image, min_crop_ratio=0.2, max_crop_ratio=0.8)

**Question 4**

Answer: Salt and pepper noise

In [19]:
def salt_and_pepper(img, noise_amt):
    noisy_img = np.copy(img)
    height, width = noisy_img.shape[:2]
    num_pixels = int(noise_amt * height * width)

    salt = np.random.randint(0, height, num_pixels), np.random.randint(0, width, num_pixels)
    noisy_img[salt] = [255,255,255]

    pepper = np.random.randint(0, height, num_pixels), np.random.randint(0, width, num_pixels)
    noisy_img[pepper] = [0,0,0]

    display_image("noisy image", noisy_img)
    
img = cv.imread("images/lena.png")
salt_and_pepper(img, 0.03)